In [42]:
import requests as r
from Cython.Plex import Actions
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

In [43]:
#Loading linkedin job search page
path = "C:/Program Files (x86)/chromedriver.exe"
driver = webdriver.Chrome(path)
driver.maximize_window()
url = "https://www.linkedin.com/jobs/search?trk=guest_homepage-basic_guest_nav_menu_jobs&position=1&pageNum=0"
driver.get(url)
time.sleep(3)

C:\Users\yashd\AppData\Local\Temp\ipykernel_22048\2461882802.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [44]:
#CLosing the sign-in pop-up
sign_in_close = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//button[@class="cta-modal__dismiss-btn absolute top-2 right-2 hover:cursor-pointer focus:outline focus:outline-2 focus:outline-color-action"]')))
sign_in_close.click()

In [45]:
#Searching for Data Science jobs in the United Arab Emirates
job_search = driver.find_element(by='name',value='keywords')
job_search.clear()
job_search.send_keys("Data Science")
job_loc = driver.find_element(By.NAME, "location")
job_loc.clear()
job_loc.send_keys("United Arab Emirates")
job_loc.send_keys(Keys.ENTER)
time.sleep(2)

In [46]:
#Entering experience filter
driver.find_element(By.XPATH,'//button[@data-tracking-control-name="public_jobs_f_E"]').click()
#Selecting intern filter
driver.find_element(By.XPATH,'//input[@id="f_E-0"]').click()
#Selecting entry level filter
driver.find_element(By.XPATH,'//input[@id="f_E-1"]').click()
#Clicking done to show results
driver.find_element(By.XPATH,'//button[@data-tracking-control-name="public_jobs_f_E"][@type="submit"]').click()

In [47]:
#Function to Scroll to end of page (See more jobs tab) to get all the top jobs
def scroll_down(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to the bottom.
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load the page.
        time.sleep(3)
        # Calculate new scroll height and compare with last scroll height.
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [48]:
#Loading all the available jobs and extracting html script of the page
time.sleep(5)
while True:
    scroll_down(driver)
    try:
        end_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//button[@class="infinite-scroller__show-more-button infinite-scroller__show-more-button--visible"]')))
        end_elem.click()
    except:
        print("All jobs loaded")
        job_html = driver.page_source
        driver.quit()
        break

All jobs loaded


In [49]:
#Making a copy of html script loaded from above and creating beautiful soup object
jobpage = job_html
soup = BeautifulSoup(jobpage,'html.parser')

In [50]:
#Functions to check for none values
def is_none_list(list):
    flag=False
    counter=0
    for i in list:
        if i is None:
            flag=True
            counter+=1
    return flag,counter

def is_none_var(var):
    if var is None:
        return True
    else:
        return False

In [51]:
#Extracting all the details of each job from job search page and storing the results in job_dict
job_table = soup.find_all('ul', class_="jobs-search__results-list")
job_dict = {}
none_counter=0
for i in job_table[0].find_all('li'):

    job_details, job_list = [], []

    company_main = i.find('h4', class_="base-search-card__subtitle" )
    company = company_main.find('a')

    job_link = i.find('a',class_="base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]")

    job_title = i.find('h3',class_="base-search-card__title").string.strip()
    job_loc = i.find('span',class_="job-search-card__location").string.strip()
    job_time = i.find('time',class_="job-search-card__listdate")

    job_list.extend([company,job_time,job_link])
    flag, msg = is_none_list(job_list)
    if flag:
        if msg == 0:
            job_details.extend([None, job_loc, job_time['datetime'], job_link['href']])
        elif msg == 1:
            job_details.extend([company.string.strip(), job_loc, None, job_link['href']])
        else:
            job_details.extend([None, None, None, None])
        none_counter+=1
        continue
    job_details.extend([company.string.strip(),job_loc,job_time['datetime'],job_link['href']])
    job_dict[job_title] = job_details
print(f"Number of jobs found: {len(job_dict)}")
print(f"Number of jobs with no data: {none_counter}")

Number of jobs found: 134
Number of jobs with no data: 13


In [52]:
#Function to convert linkedin page to english
def convert_to_eng(url):
    driver = webdriver.Chrome(path)
    driver.get(url)
    #driver.get(url)
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    dropdown = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,'//button[@data-tracking-control-name="footer-lang-dropdown_trigger"]')))
    dropdown.click()
    lang = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,'//button[@data-tracking-control-name="language-selector-en_US"]')))
    lang.click()
    time.sleep(2)
    script = driver.page_source
    driver.quit()
    return script

In [ ]:
job_dict

In [53]:
#Visiting listing of each job to extract further details and storing in job_details_dict
job_details_dict = {}
for job_title, job_details in job_dict.items():

    #Checking if job details is none
    if job_details[-1] is None:
        job_details_dict[job_title] = [None, None, None, None]
        continue

    #Converting language of page from arabic to english
    job_page = convert_to_eng(job_dict[job_title][-1])
    soup = BeautifulSoup(job_page, 'html.parser')
    criteria = soup.find('ul', class_="description__job-criteria-list")
    if is_none_var(criteria):
        job_details_dict[job_title] = [None,None,None, None]
        continue
    criteria_list = []
    for i in criteria.find_all('li',class_="description__job-criteria-item"):
        v = i.find('span').string.strip()
        criteria_list.append(v)
    job_details_dict[job_title] = criteria_list

driver.quit()
print(len(job_details_dict) == len(job_dict))

C:\Users\yashd\AppData\Local\Temp\ipykernel_22048\2506048946.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


True


In [ ]:
job_details_dict

In [54]:
#Creating dataframe from job_dict dictionary
list = []
for key, value in job_details_dict.items():
    l=[]
    l.append(key)
    for v in value:
        l.append(v)
    list.append(l)
job_pd_1 = pd.DataFrame(data=list, columns=['Job Title','Seniority Level','Employment Type','Job Function','Industries'])

In [55]:
#Creating Dataframe from job_details_dict
list=[]
for key,value in job_dict.items():
    l=[key]
    if value == "None":
        none_list = ("None")
    for i in range(len(value)-1):
      l.append(value[i])
    list.append(l)
job_pd_2 = pd.DataFrame(list,columns=['Job Title', 'Company','Location','Date of listing'])

In [56]:
#Merging the two dataframes using Job Title as key
job_pd = pd.merge(left=job_pd_2, right=job_pd_1, on=['Job Title'])
job_pd

,Job Title,Company,Location,Date of listing,Seniority Level,Employment Type,Job Function,Industries
0,Data Scientist,Halian,"Abu Dhabi, Abu Dhabi Emirate, United Arab Emir...",2022-11-28,Entry level,Contract,Engineering and Information Technology,IT Services and IT Consulting
1,data scientist - 1283651,Halian,United Arab Emirates,2023-01-31,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting
2,Data and Business Analyst,MedSupply International,"Dubai, United Arab Emirates",2023-01-04,None,None,None,None
3,Data Analyst / Data Scientist (Remote),nybl,"Dubai, Dubai, United Arab Emirates",2022-12-07,Entry level,Full-time,Other,IT Services and IT Consulting
4,Data Engineer Intern,dubizzle,"Dubai, Dubai, United Arab Emirates",2023-01-17,Internship,Full-time,Information Technology,Online Audio and Video Media
...,...,...,...,...,...,...,...,...
129,Post-Doctoral Associate in the Center for Quan...,New York University Abu Dhabi,"Abu Dhabi, Abu Dhabi Emirate, United Arab Emir...",2023-01-26,None,None,None,None
130,Post-Doctoral Associate in the Division of Soc...,New York University Abu Dhabi,"Abu Dhabi, Abu Dhabi Emirate, United Arab Emir...",2023-01-21,None,None,None,None
131,Part-time Table Tennis Coach (Female),Ajman University,"Ajman, Ajman Emirate, United Arab Emirates",2023-01-07,Entry level,Part-time,Education and Training,Higher Education
132,Commis(Chinese Cuisine),Kitopi,"Abu Dhabi, Abu Dhabi Emirate, United Arab Emir...",2023-01-23,Entry level,Full-time,Management and Manufacturing,Food & Beverages


In [57]:
job_pd.to_csv('linkedin_job_listings.csv', index=False)